# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction: Business Problem <a name="introduction"></a>

In this project, we will investigate the influence of poverty rate to the facilities and amenities of certain borough and how this insights can provide targeting local and business development for London boroughs. It will be interesting to see how poverty impacts the facilities, venues and amenities available in the boroughs. 

<br><br>
<center><img src="poverty_rate.jpg"
     alt="London Poverty Rates"
     width = "700"
     style="float: centre; margin-left: 10px;" /></center><br><br>


This will be done by carrying out a comparative studies on the facilities and amenities between the two boroughs with the highest income rate and the lowest in London, **Tower Hamlet (TH)** borough and **Bromley (Brom)** borough of London, as shown in the figure above.  

We will use the data science principles and techniques learned to generate a model of these boroughs, looking at the venues and facilities available in these areas and provides insights to stakeholders, the local authorities and business chambers of commerce.

### Questions to answer:

1.	What types of facilities (venues) and amenities are available in the area with different poverty line?
2.	How venues changing based on the spending power?
3.	What are the distinctive venues that represent in these boroughs?
4.	Suggestions and recommendations. 

By answering the above questions, the findings can be used for the targeting development for the rest of the London Boroughs so that unnecessary development can be avoided and overall budget can be sustained.

======================================================================================================================== 

## 2. Data <a name="data"></a>

Based on the problem definition, factors that will influence the decision in this project will be:

* number and the type of venues and facilities available in the suurounding area of these boroughs
* the most frequent venues and facilties for each boroughs

To define the surrounding area of the borough, we will be using:
* London Boroughs poverty information from: **https://www.trustforlondon.org.uk/data/poverty-borough**. *Accessed: 11/03/2019*
* latitudes and longitudes of Tower Hamlet and Bromley  obtained from: **https://www.distancesto.com/coordinates/gb/**. *Accessed: 11/03/2019*  
* venues, type and locations in every borough will be obtained using **Foursquare API**

Further information of the boroughs can be found at:
* Bromley: **https://en.wikipedia.org/wiki/London_Borough_of_Bromley** 
* Tower Hamlet: **https://en.wikipedia.org/wiki/London_Borough_of_Tower_Hamlets**

### Boroughs Locations 
Based on the information obtained from https://www.distancesto.com/coordinates/gb/, the latitude and the longitude of TM and Brom are as follows:

In [2]:
TM_coordinates = (51.520261, -0.02934)
BROM_coordinates = (51.367971, 0.070062)
london_coordinates = (51.509865, -0.118092)

Let's visualise the locations of these boroughs on London map:

In [21]:
import folium
from folium.features import DivIcon

london_map = folium.Map(location = london_coordinates, zoom_start = 10)

folium.Circle(
    radius=2500,
    location= TM_coordinates,
    popup = ('Tower Hamlet'),
    color= 'crimson',
    fill=False,
).add_to(london_map)

folium.Circle(
    radius=6900,
    location= BROM_coordinates,
    popup = 'Bromley',
    color='blue',
    fill=False,
).add_to(london_map)


london_map

Preliminary observations from the above map show that based on the locations for the two boroughs, Tower Hamlet is located very near to the London centre, where as Bromley borough located at the boundary of the M25 Ringroad, which is about 2 hours drive from London centre. In terms of area size, Tower Hamlet convers about 19.77km2 and Bromley is about 150.2km2. Based on these information, we can used them as distance references when retrieving venues, type and locations  using Foursquare API.

### Foursquare API

Now that we have our location candidates, let's use Foursquare API to get info on venues in each of the borough

As an exploratory project, we will retrieve the venues based on the areas of each borough. We will then do the neccesary manipulations and analysis to achieve our objectives.  

======================================================================================================================== 

## 3. Methodology <a name="methodology"></a>

======================================================================================================================== 

## 4. Analysis <a name="analysis"></a>

======================================================================================================================== 

## 5. Results and Discussion <a name="results"></a>

======================================================================================================================== 

## 6. Conclusions <a name="conclusion"></a>

======================================================================================================================== 